In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json
import os
import hashlib
import re

options=webdriver.ChromeOptions()
options.add_argument("--incognito")

driver=webdriver.Chrome()
driver.set_window_size(1200,960)

listData=[]
listLink=[]
listChapterTitleURL=[]
listChapter=[]

def visit():
    
    driver.get('https://www.bookwormzz.com/zh/')
    sleep(3)

def url():
    
#取得每冊"URL"
    if len(driver.find_elements(By.CSS_SELECTOR, "a.ui-btn.ui-btn-icon-right.ui-icon-carat-r")) > 0:
        a=driver.find_elements(By.CSS_SELECTOR, "a.ui-btn.ui-btn-icon-right.ui-icon-carat-r")
        for index2, subook in enumerate(a):
            if index2 >= 46:
                break
            aLink =subook.get_attribute('href')
            print("取得網址: {}".format(aLink))
            listLink.append(aLink)
#取得每冊每回章節"URL"和"標題"
    for aLink in listLink:
        driver.get(aLink)
        sleep(1)
        driver.find_element(By.CSS_SELECTOR,'a.ui-link.ui-btn.ui-icon-bullets.ui-btn-icon-left').click()
        sleep(1)
        contentlinks=driver.find_elements(By.CSS_SELECTOR,'div.ui-content a.ui-link')
        for  chapterlink in contentlinks:
            title=chapterlink.text
#             print(title)
            chapterurl=chapterlink.get_attribute("href")
#             print(chapterurl)
            listChapterTitleURL.append({
                "Title":title,
                "URL":chapterurl
            })
               
def parse():

#  每冊的目錄頁    
    for aLink in listLink:
        driver.get(aLink)
        sleep(3)
        eChapter=driver.find_element(By.CSS_SELECTOR,'div.ui-content div').text
        subookT=driver.find_element(By.CSS_SELECTOR,' h1.title.ui-title').text
        subooktitle=re.search(r"(.+) 線上小說閱讀", subookT)[1]
        listChapter.append({
            "subBookTitle":subooktitle,
            "Chapter":eChapter})
        
# 每冊目錄存成text檔          
    for obj in listChapter:
        #建立相應名稱資料夾
        folder="D:\\jupyter_notes\\蟲蟲爬金庸改\\{}".format(obj["subBookTitle"])
        os.mkdir(folder)
        #把text檔輸出到指定資料夾
        fp = open("{}\\{}目錄.txt".format(folder, obj["subBookTitle"]), "w", encoding='utf-8')
        fp.write(obj["Chapter"])
        fp.close()
        
#  每冊每回的章節內容
    for chapter in listChapterTitleURL:
        chapterTitle=chapter["Title"]
        driver.get(chapter["URL"])
        sleep(1)
        chapterContent=driver.find_element(By.CSS_SELECTOR,'div.ui-content div').text
        subookT=driver.find_element(By.CSS_SELECTOR,' h1.title.ui-title').text
        folder=re.search(r"(.+) 線上小說閱讀", subookT)[1]
        listData.append({
                "folderName":folder,
                "chaptertitle":chapterTitle,
                "chaptercontent":chapterContent
            })
        
# 每回存成text檔                 
    for obj in listData:
         #相應資料夾名稱
        folder="D:\\jupyter_notes\\蟲蟲爬金庸改\\{}".format(obj["folderName"]) 
         #把text檔輸出到指定資料夾
        fp = open("{}\\{}.txt".format(folder, obj["chaptertitle"]), "w", encoding='utf-8')
        fp.write(obj["chaptercontent"])
        fp.close()

# def saveJson():
#     for obj in listData:
#         fp = open(obj["title"]+".json", "w", encoding='utf-8')
#         fp.write(json.dumps(obj["content"], ensure_ascii=False))
#         fp.close()

        
# 主程式執行
visit()
url()
parse()
# saveJson()





WebDriverException: Message: chrome not reachable
  (Session info: chrome=83.0.4103.116)
